# Beam conversion from Bigquery to TF Records

In this notebook we use Apache Beam to convert to tfrecords
The applications can be found in `beam_candidates` and `beam_training` for candidate generation and training

## pip & package

In [1]:
# !pip install --upgrade 'apache-beam[gcp]' --user

In [2]:
import os
import time
from pprint import pprint

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

import tensorflow as tf

from google.cloud import storage

### Set variables

In [3]:
PREFIX = 'twotower-v2'

In [4]:
# staging GCS
GCP_PROJECTS             = !gcloud config get-value project
PROJECT_ID               = GCP_PROJECTS[0]

# GCS bucket and paths
BUCKET_NAME              = f'{PREFIX}-{PROJECT_ID}-bucket'
BUCKET_URI               = f'gs://{BUCKET_NAME}'

config = !gsutil cat {BUCKET_URI}/config/notebook_env.py
print(config.n)
exec(config.n)


PROJECT_ID               = "hybrid-vertex"
PROJECT_NUM              = "934903580331"
LOCATION                 = "us-central1"

REGION                   = "us-central1"
BQ_LOCATION              = "US"
VPC_NETWORK_NAME         = "ucaip-haystack-vpc-network"

VERTEX_SA                = "934903580331-compute@developer.gserviceaccount.com"

PREFIX                   = "twotower-v2"
VERSION                  = "v2"
DATA_VERSION             = "v3data"

BUCKET_NAME              = "twotower-v2-hybrid-vertex-bucket"
BUCKET_URI               = "gs://twotower-v2-hybrid-vertex-bucket"
SOURCE_BUCKET            = "spotify-million-playlist-dataset"

DATA_GCS_PREFIX          = "data"
DATA_PATH                = "gs://twotower-v2-hybrid-vertex-bucket/data"
VOCAB_SUBDIR             = "vocabs"
VOCAB_FILENAME           = "vocab_dict.pkl"

VPC_NETWORK_FULL         = "projects/934903580331/global/networks/ucaip-haystack-vpc-network"

BQ_DATASET               = "spotify_e2e_test"

REPO_SRC                 = "sr

In [5]:
# GCP_PROJECTS = !gcloud config get-value project
# PROJECT_ID = GCP_PROJECTS[0]

In [6]:
storage_client = storage.Client(project=PROJECT_ID)

# TODO - parameterize
# REGION = 'us-central1'
# NETWORK = 'ucaip-haystack-vpc-network'
# BUCKET_NAME = 'matching-engine-content'


BQ_DATASET = 'spotify_e2e_test'
BQ_TABLE_TRAIN = 'v2_train_flatten_last_5' # TODO
BQ_TABLE_VALID = 'v2_train_flatten_valid_last_5'
BQ_TABLE_CANDIDATES = 'candidates'

MAX_PLAYLIST_LENGTH = 5

# VERSION = "v2-0-0" # version tag for dataflow pipeline

# gcs subfolders - destinations for processed data: f'gs://{BUCKET_NAME}/{VERSION}/{___PREFIX}'
CANDIDATE_PREFIX = 'candidates'
TRAIN_DIR_PREFIX = 'train'
VALID_DIR_PREFIX = 'valid'

In [7]:
! gsutil ls -al gs://$BUCKET_NAME

                                 gs://twotower-v2-hybrid-vertex-bucket/config/


# Run Dataflow to convert BQ to TFrecords

Candidate generation can be found in `beam_candidates`
Training and Validation generation can be found in `beam_training`

Usage:
* Candidate generation

> `beam_candidates\python3 main.py $PROJECT_ID $NETWORK $REGION $VERSION $BUCKET_NAME $CANDIDATE_PREFIX $BQ_DATASET $BQ_TABLE_CANDIDATES`
   
* Training generation
  
> `beam_training\python3 main-train.py <BQ_table> <gcs data subfolder> <desired partition size MB> <BQ dataset size MB> <version tag>`

In [8]:
!tree beam_training

beam_training
├── README.MD
├── __init__.py
├── create_tfrecords_training.egg-info
│   ├── PKG-INFO
│   ├── SOURCES.txt
│   ├── dependency_links.txt
│   ├── requires.txt
│   └── top_level.txt
├── main-train.py
├── setup.py
└── train_pipeline
    ├── __init__.py
    ├── __pycache__
    │   ├── __init__.cpython-37.pyc
    │   ├── train_pipe.cpython-37.pyc
    │   └── train_pipe_shape.cpython-37.pyc
    └── train_pipe_shape.py

3 directories, 14 files


In [9]:
# os.getcwd()

In [10]:
%cd beam_candidates

/home/jupyter/jw-repo2/spotify_mpd_two_tower/beam_candidates


### Candidates

In [11]:
start_time = time.time()

! python3 main.py $PROJECT_ID $VPC_NETWORK_NAME $REGION $DATA_VERSION $BUCKET_NAME $CANDIDATE_PREFIX $BQ_DATASET $BQ_TABLE_CANDIDATES

end_time = time.time()
runtime_mins = int((end_time - start_time) / 60)
print(f"total runtime_mins: {runtime_mins}")

GoogleCloudOptions(create_from_snapshot=None, dataflow_endpoint=https://dataflow.googleapis.com, dataflow_kms_key=None, dataflow_service_options=None, enable_artifact_caching=False, enable_hot_key_logging=False, enable_streaming_engine=False, flexrs_goal=None, gcp_oauth_scopes=['https://www.googleapis.com/auth/bigquery', 'https://www.googleapis.com/auth/cloud-platform', 'https://www.googleapis.com/auth/devstorage.full_control', 'https://www.googleapis.com/auth/userinfo.email', 'https://www.googleapis.com/auth/datastore', 'https://www.googleapis.com/auth/spanner.admin', 'https://www.googleapis.com/auth/spanner.data'], impersonate_service_account=None, job_name=spotify-bq-tfrecords-candidates-v3data-230919-024633, labels=None, no_auth=False, project=hybrid-vertex, region=us-central1, service_account_email=None, staging_location=gs://twotower-v2-hybrid-vertex-bucket/v3data/job/staging/, temp_location=gs://twotower-v2-hybrid-vertex-bucket/v3data/job/temp/, template_location=None, transform

In [12]:
!pwd

/home/jupyter/jw-repo2/spotify_mpd_two_tower/beam_candidates


In [13]:
%cd ..

/home/jupyter/jw-repo2/spotify_mpd_two_tower


### Validation set

In [14]:
%cd beam_training

/home/jupyter/jw-repo2/spotify_mpd_two_tower/beam_training


In [15]:
TARGET_SHARD_SIZE_MB_VALID = 250
TOTAL_MB_VALID = 500
NUM_TF_RECORDS = int(TOTAL_MB_VALID) // int(TARGET_SHARD_SIZE_MB_VALID)
NUM_TF_RECORDS

2

In [16]:
start_time = time.time()

# ! python3 main-train.py $BQ_TABLE_VALID $VALID_DIR_PREFIX $TARGET_SHARD_SIZE_MB_VALID $TOTAL_MB_VALID $VERSION $BUCKET_NAME $REGION $PROJECT_ID $NETWORK $BQ_DATASET

! python3 main-train.py $PROJECT_ID $VPC_NETWORK_NAME $REGION $DATA_VERSION $BUCKET_NAME $VALID_DIR_PREFIX $TOTAL_MB_VALID $TARGET_SHARD_SIZE_MB_VALID $BQ_DATASET $BQ_TABLE_VALID

end_time = time.time()
runtime_mins = int((end_time - start_time) / 60)
print(f"total runtime_mins: {runtime_mins}")

Number of Expected TFRecords: 2
GoogleCloudOptions(create_from_snapshot=None, dataflow_endpoint=https://dataflow.googleapis.com, dataflow_kms_key=None, dataflow_service_options=None, enable_artifact_caching=False, enable_hot_key_logging=False, enable_streaming_engine=False, flexrs_goal=None, gcp_oauth_scopes=['https://www.googleapis.com/auth/bigquery', 'https://www.googleapis.com/auth/cloud-platform', 'https://www.googleapis.com/auth/devstorage.full_control', 'https://www.googleapis.com/auth/userinfo.email', 'https://www.googleapis.com/auth/datastore', 'https://www.googleapis.com/auth/spanner.admin', 'https://www.googleapis.com/auth/spanner.data'], impersonate_service_account=None, job_name=spotify-bq-tfrecords-v3data-230919-030652, labels=None, no_auth=False, project=hybrid-vertex, region=us-central1, service_account_email=None, staging_location=gs://twotower-v2-hybrid-vertex-bucket/v3data/job/staging/, temp_location=gs://twotower-v2-hybrid-vertex-bucket/v3data/job/temp/, template_loc

### Tain set

In [17]:
TARGET_SHARD_SIZE_MB_TRAIN = 2000
TOTAL_MB_TRAIN = 44_000
NUM_TF_RECORDS = int(TOTAL_MB_TRAIN) // int(TARGET_SHARD_SIZE_MB_TRAIN)
NUM_TF_RECORDS

22

In [ ]:
start_time = time.time()

! python3 main-train.py $PROJECT_ID $VPC_NETWORK_NAME $REGION $DATA_VERSION $BUCKET_NAME $TRAIN_DIR_PREFIX $TOTAL_MB_TRAIN $TARGET_SHARD_SIZE_MB_TRAIN $BQ_DATASET $BQ_TABLE_TRAIN

end_time = time.time()
runtime_mins = int((end_time - start_time) / 60)
print(f"total runtime_mins: {runtime_mins}")

Number of Expected TFRecords: 22
GoogleCloudOptions(create_from_snapshot=None, dataflow_endpoint=https://dataflow.googleapis.com, dataflow_kms_key=None, dataflow_service_options=None, enable_artifact_caching=False, enable_hot_key_logging=False, enable_streaming_engine=False, flexrs_goal=None, gcp_oauth_scopes=['https://www.googleapis.com/auth/bigquery', 'https://www.googleapis.com/auth/cloud-platform', 'https://www.googleapis.com/auth/devstorage.full_control', 'https://www.googleapis.com/auth/userinfo.email', 'https://www.googleapis.com/auth/datastore', 'https://www.googleapis.com/auth/spanner.admin', 'https://www.googleapis.com/auth/spanner.data'], impersonate_service_account=None, job_name=spotify-bq-tfrecords-v3data-230919-032310, labels=None, no_auth=False, project=hybrid-vertex, region=us-central1, service_account_email=None, staging_location=gs://twotower-v2-hybrid-vertex-bucket/v3data/job/staging/, temp_location=gs://twotower-v2-hybrid-vertex-bucket/v3data/job/temp/, template_lo

# Test output

In [17]:
# import utils
# from src.two_tower_jt import feature_sets
from util import feature_sets

## Candidates

### Candidate tower features

In [18]:
candidate_features = feature_sets.get_candidate_features()
candidate_features

{'track_uri_can': FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'track_name_can': FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'artist_uri_can': FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'artist_name_can': FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'album_uri_can': FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'album_name_can': FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'duration_ms_can': FixedLenFeature(shape=(), dtype=tf.float32, default_value=None),
 'track_pop_can': FixedLenFeature(shape=(), dtype=tf.float32, default_value=None),
 'artist_pop_can': FixedLenFeature(shape=(), dtype=tf.float32, default_value=None),
 'artist_genres_can': FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'artist_followers_can': FixedLenFeature(shape=(), dtype=tf.float32, default_value=None),
 'track_danceability_can': FixedLenFeature(shape=(), dtype=tf.float32, defa

### Candidate files

In [19]:
candidate_files = []

for blob in storage_client.list_blobs(f"{BUCKET_NAME}", prefix=f'{DATA_VERSION}/{CANDIDATE_PREFIX}'):
    candidate_files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))

candidate_dataset = tf.data.TFRecordDataset(candidate_files)

parsed_candidate_dataset = candidate_dataset.map(feature_sets.parse_candidate_tfrecord_fn)

In [20]:
for x in parsed_candidate_dataset.batch(1).take(1):
    pprint(x)

{'album_name_can': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Fallen'], dtype=object)>,
 'album_uri_can': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'spotify:album:20mh1X1FQQidKMu2XypyMI'], dtype=object)>,
 'artist_followers_can': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([3735081.], dtype=float32)>,
 'artist_genres_can': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b"'alternative metal'"], dtype=object)>,
 'artist_name_can': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Evanescence'], dtype=object)>,
 'artist_pop_can': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([74.], dtype=float32)>,
 'artist_uri_can': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'spotify:artist:5nGIFgo0shDenQYSE0Sn7c'], dtype=object)>,
 'duration_ms_can': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([278066.], dtype=float32)>,
 'track_acousticness_can': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.00558], dtype=float32)>,
 'track_danceability_can

## Train & Valid datasets

In [21]:
MAX_PLAYLIST_LENGTH = 5

### train & valid features

In [22]:
feats = feature_sets.get_all_features(MAX_PLAYLIST_LENGTH)
feats

{'track_uri_can': FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'track_name_can': FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'artist_uri_can': FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'artist_name_can': FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'album_uri_can': FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'album_name_can': FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'duration_ms_can': FixedLenFeature(shape=(), dtype=tf.float32, default_value=None),
 'track_pop_can': FixedLenFeature(shape=(), dtype=tf.float32, default_value=None),
 'artist_pop_can': FixedLenFeature(shape=(), dtype=tf.float32, default_value=None),
 'artist_genres_can': FixedLenFeature(shape=(), dtype=tf.string, default_value=None),
 'artist_followers_can': FixedLenFeature(shape=(), dtype=tf.float32, default_value=None),
 'track_danceability_can': FixedLenFeature(shape=(), dtype=tf.float32, defa

### Valid files

In [23]:
valid_files = []
for blob in storage_client.list_blobs(f"{BUCKET_NAME}", prefix=f'{DATA_VERSION}/{VALID_DIR_PREFIX}/'):
    if '.tfrecords' in blob.name:
        valid_files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))
    
valid = tf.data.TFRecordDataset(valid_files)

valid_parsed = valid.map(feature_sets.parse_tfrecord)
# valid_parsed

In [24]:
for x in valid_parsed.batch(1).take(1):
    print(x)

{'album_name_can': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'digital druglord'], dtype=object)>, 'album_name_pl': <tf.Tensor: shape=(1, 5), dtype=string, numpy=
array([[b'Grateful', b'Despacito Feat. Justin Bieber',
        b'Tormenta Tropical Vol. 1', b'Nine Track Mind',
        b'Take A Step Back']], dtype=object)>, 'album_uri_can': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'spotify:album:30QoUTsSdpQuZunPNFsLnm'], dtype=object)>, 'album_uri_pl': <tf.Tensor: shape=(1, 5), dtype=string, numpy=
array([[b'spotify:album:4JBZ0QHveEpESepanNBG8A',
        b'spotify:album:3smvpv7CdrhVcGYaNDLOqn',
        b'spotify:album:06NtaVdvkbmGF2EEontxzt',
        b'spotify:album:5J4SS8wTmXdyIEVYjmHzpZ',
        b'spotify:album:5JqlAl0bi9cnORNubLKQVI']], dtype=object)>, 'artist_followers_can': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([4841523.], dtype=float32)>, 'artist_genres_can': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b"'electropop', 'pop', 'pop rap'"], dtype

### Train files

In [25]:
train_files = []
for blob in storage_client.list_blobs(f"{BUCKET_NAME}", prefix=f'{DATA_VERSION}/{TRAIN_DIR_PREFIX}/', delimiter="/"):
    if '.tfrecords' in blob.name:
        train_files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))
    
train = tf.data.TFRecordDataset(train_files)

train_parsed = train.map(feature_sets.parse_tfrecord)

In [26]:
train_parsed

<MapDataset element_spec={'album_name_can': TensorSpec(shape=(), dtype=tf.string, name=None), 'album_name_pl': TensorSpec(shape=(5,), dtype=tf.string, name=None), 'album_uri_can': TensorSpec(shape=(), dtype=tf.string, name=None), 'album_uri_pl': TensorSpec(shape=(5,), dtype=tf.string, name=None), 'artist_followers_can': TensorSpec(shape=(), dtype=tf.float32, name=None), 'artist_genres_can': TensorSpec(shape=(), dtype=tf.string, name=None), 'artist_genres_pl': TensorSpec(shape=(5,), dtype=tf.string, name=None), 'artist_name_can': TensorSpec(shape=(), dtype=tf.string, name=None), 'artist_name_pl': TensorSpec(shape=(5,), dtype=tf.string, name=None), 'artist_pop_can': TensorSpec(shape=(), dtype=tf.float32, name=None), 'artist_pop_pl': TensorSpec(shape=(5,), dtype=tf.float32, name=None), 'artist_uri_can': TensorSpec(shape=(), dtype=tf.string, name=None), 'artist_uri_pl': TensorSpec(shape=(5,), dtype=tf.string, name=None), 'artists_followers_pl': TensorSpec(shape=(5,), dtype=tf.float32, name

In [27]:
for x in train_parsed.batch(1).take(1):
    print(x)

{'album_name_can': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'My Worlds - The Collection'], dtype=object)>, 'album_name_pl': <tf.Tensor: shape=(1, 5), dtype=string, numpy=
array([[b'My World 2.0', b'Stronger', b'Jason Derulo', b'Jeremih',
        b'PCD']], dtype=object)>, 'album_uri_can': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'spotify:album:57rkSLAFTlhYM7JcQc9arl'], dtype=object)>, 'album_uri_pl': <tf.Tensor: shape=(1, 5), dtype=string, numpy=
array([[b'spotify:album:3BmcYMh0KYsimWL6p2gPa9',
        b'spotify:album:5bKKfWulVeKWFd80Dwr3Kx',
        b'spotify:album:0aVJmVAeEx78nAA1rAKYf7',
        b'spotify:album:5CKAAsmFCdmKq4fTtNdmHc',
        b'spotify:album:0vHdsDCKhWTVe6GjvgLQOC']], dtype=object)>, 'artist_followers_can': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([70336320.], dtype=float32)>, 'artist_genres_can': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b"'canadian pop', 'pop'"], dtype=object)>, 'artist_genres_pl': <tf.Tensor: shape=(1, 5),

**Finished**